In [1]:
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
from course_intro_ocr_t1.data import MidvPackage
from tqdm import tqdm
from matplotlib import pyplot as plt
import numpy as np

import albumentations as albu

from iglovikov_helper_functions.utils.image_utils import load_rgb, pad, unpad
from iglovikov_helper_functions.dl.pytorch.utils import tensor_from_rgb_image

import torch

import cv2 
from PIL import Image

from midv500models.pre_trained_models import create_model

# Загрузка данных

In [2]:
DATASET_PATH = Path().absolute().parent.parent /  'midv500_compressed'
assert DATASET_PATH.exists(), DATASET_PATH.absolute()

data_packs = MidvPackage.read_midv500_dataset(DATASET_PATH)
len(data_packs), type(data_packs[0])

(50, course_intro_ocr_t1.data.MidvPackage)

# Основная Часть
Была использована предобученная модель на основе UNET.

In [4]:
model = create_model("Unet_resnet34_2020-05-19")
model.eval()

def prep(image):
    image = np.array(image)
    transform = albu.Compose([albu.Normalize(p=1)], p=1)
    padded_image, pads = pad(image, factor=32, border=cv2.BORDER_CONSTANT)

    x = transform(image=padded_image)["image"]
    x = torch.unsqueeze(tensor_from_rgb_image(x), 0)
    return x, pads

In [5]:
def post(prediction, pads):
    mask = (prediction > 0).cpu().numpy().astype(np.uint8)
    mask = unpad(mask, pads)
    
    for i in range(0,800):
        for j in range(0,450):
            if(mask[i][j] > 0 ):
                mask[i][j] = 255
    ret, threshold = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        epsilon = 0.04 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
    
    if len(approx) == 4:
        white_figure_coordinates = approx

    white_figure_coordinates  = np.array(white_figure_coordinates)
    white_figure_coordinates  = white_figure_coordinates.astype(np.float64)

    for i in range(4):
        for j in range(2):
            if (j == 0):
                white_figure_coordinates[i][0][j] = float(white_figure_coordinates[i][0][j]) / 450.0
            else:
                white_figure_coordinates[i][0][j] = float(white_figure_coordinates[i][0][j]) / 800.0
    
    white_figure_coordinates = white_figure_coordinates.reshape(4,2)
    return white_figure_coordinates


In [6]:
results_dict = dict()

for dp in tqdm(data_packs):
    for i in range(len(dp)):
        if dp[i].is_test_split():
            try:
                x, pads = prep(dp[i].image)
                with torch.no_grad():
                    prediction = model(x)[0][0]
                results_dict[dp[i].unique_key] = post(prediction, pads)
            except Exception as exc:
                # Для пропущенных в словаре ключей в метриках автоаматически засчитается IoU=0
                print(exc)

  2%|▉                                           | 1/50 [00:51<41:45, 51.13s/it]

local variable 'white_figure_coordinates' referenced before assignment
local variable 'white_figure_coordinates' referenced before assignment
local variable 'white_figure_coordinates' referenced before assignment
local variable 'white_figure_coordinates' referenced before assignment


  4%|█▋                                       | 2/50 [06:19<2:51:15, 214.08s/it]

local variable 'approx' referenced before assignment
local variable 'approx' referenced before assignment
local variable 'approx' referenced before assignment
local variable 'approx' referenced before assignment
local variable 'approx' referenced before assignment
local variable 'approx' referenced before assignment
local variable 'approx' referenced before assignment
local variable 'approx' referenced before assignment
local variable 'approx' referenced before assignment
local variable 'approx' referenced before assignment


  6%|██▍                                      | 3/50 [11:45<3:27:55, 265.43s/it]

...

 90%|████████████████████████████████████▉    | 45/50 [1:14:01<04:30, 54.19s/it]

local variable 'white_figure_coordinates' referenced before assignment


 92%|█████████████████████████████████████▋   | 46/50 [1:14:56<03:37, 54.32s/it]

local variable 'white_figure_coordinates' referenced before assignment
local variable 'white_figure_coordinates' referenced before assignment


100%|█████████████████████████████████████████| 50/50 [1:18:33<00:00, 94.28s/it]


In [16]:
dump_results_dict(results_dict, Path() / 'pred.json')